In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

import os
os.chdir('/content/gdrive/MyDrive/8 魯迅チャットボット/')

Mounted at /content/gdrive/


# 対話の検証

## １文字の読み込み

In [2]:
import pickle

with open('kana_chars.pickle', mode='rb') as f:
    chars_list = pickle.load(f)
print(chars_list)

['\t', '\n', '!', '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'C', 'M', 'N', 'O', 'P', 'Q', 'a', 'd', 'e', 'f', 'h', 'i', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'y', '×', '―', '○', '、', '。', '々', '〆', '〇', 'ぁ', 'あ', 'ぃ', 'い', 'ぅ', 'う', 'ぇ', 'え', 'ぉ', 'お', 'か', 'が', 'き', 'ぎ', 'く', 'ぐ', 'け', 'げ', 'こ', 'ご', 'さ', 'ざ', 'し', 'じ', 'す', 'ず', 'せ', 'ぜ', 'そ', 'ぞ', 'た', 'だ', 'ち', 'ぢ', 'っ', 'つ', 'づ', 'て', 'で', 'と', 'ど', 'な', 'に', 'ぬ', 'ね', 'の', 'は', 'ば', 'ぱ', 'ひ', 'び', 'ぴ', 'ふ', 'ぶ', 'ぷ', 'へ', 'べ', 'ぺ', 'ほ', 'ぼ', 'ぽ', 'ま', 'み', 'む', 'め', 'も', 'ゃ', 'や', 'ゅ', 'ゆ', 'ょ', 'よ', 'ら', 'り', 'る', 'れ', 'ろ', 'ゎ', 'わ', 'ゐ', 'ゑ', 'を', 'ん', 'ゝ', 'ゞ', 'ァ', 'ア', 'ィ', 'イ', 'ゥ', 'ウ', 'ェ', 'エ', 'ォ', 'オ', 'カ', 'ガ', 'キ', 'ギ', 'ク', 'グ', 'ケ', 'ゲ', 'コ', 'ゴ', 'サ', 'ザ', 'シ', 'ジ', 'ス', 'ズ', 'セ', 'ゼ', 'ソ', 'ゾ', 'タ', 'ダ', 'チ', 'ヂ', 'ッ', 'ツ', 'ヅ', 'テ', 'デ', 'ト', 'ド', 'ナ', 'ニ', 'ヌ', 'ネ', 'ノ', 'ハ', 'バ', 'パ', 'ヒ', 'ビ', 'ピ', 'フ', 'ブ', 'プ', 'ヘ', 'ベ', 'ペ', 'ホ', 'ボ', 'ポ', 'マ', 'ミ', 'ム', 'メ', 'モ', 'ャ

## ２文章のベクトル化

In [3]:
import numpy as np

# インデックスと文字で辞書を作成
char_indices = {}
for i, char in enumerate(chars_list):
    char_indices[char] = i
indices_char = {}
for i, char in enumerate(chars_list):
    indices_char[i] = char
    
n_char = len(chars_list)
max_length_x = 128

# 文章をone-hot表現に変換する関数
def sentence_to_vector(sentence):
    vector = np.zeros((1, max_length_x, n_char), dtype=np.bool)
    for j, char in enumerate(sentence):
        vector[0][j][char_indices[char]] = 1
    return vector

## ３返答作成用の関数

In [4]:
from keras.models import load_model

encoder_model = load_model('encoder_model.h5')
decoder_model = load_model('decoder_model.h5')

def respond(input_data, beta=5):
    state_value = encoder_model.predict(input_data)
    y_decoder = np.zeros((1, 1, n_char))  # decoderの出力を格納する配列
    y_decoder[0][0][char_indices['\t']] = 1  # decoderの最初の入力はタブ。one-hot表現にする。

    respond_sentence = ""  # 返答の文字列
    while True:
        y, h = decoder_model.predict([y_decoder, state_value])
        p_power = y[0][0] ** beta  # 確率分布の調整
        next_index = np.random.choice(len(p_power), p=p_power/np.sum(p_power)) 
        next_char = indices_char[next_index]  # 次の文字
        
        if (next_char == "\n" or len(respond_sentence) >= max_length_x):
            break  # 次の文字が改行のとき、もしくは最大文字数を超えたときは終了
            
        respond_sentence += next_char
        y_decoder = np.zeros((1, 1, n_char))  # 次の時刻の入力
        y_decoder[0][0][next_index] = 1

        state_value = h  # 次の時刻の状態

    return respond_sentence

## ４対話生成の検証

In [9]:
sentences = ["ろじんさん、こんにちは。",
             "カムパネラってしってますか？",
             "ああ、おなかすいた。",
             "きょうはいいてんきですね。",
             "すきなたべものは、なんですか。",
             "きょうは、さんぽにいきました。",
             "パソコンかスマホは、つかったことありますか？"]

for sentence in sentences:
    vector = sentence_to_vector(sentence)
    print("Input:", sentence)
    print("Response", respond(vector))
    print()

Input: ろじんさん、こんにちは。
1/1 [==============================] - 0s 16ms/step


<ipython-input-3-891c0e32d576>:16: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  vector = np.zeros((1, max_length_x, n_char), dtype=np.bool)


1/1 [==============================] - 0s 20ms/step
Response あたまのなかにはいつものとうりにいった。

Input: カムパネラってしってますか？
1/1 [==============================] - 0s 21ms/step
Response こんなにこしをおろしているので、かれはこどもをかいてあるきだした。

Input: ああ、おなかすいた。
1/1 [==============================] - 0s 20ms/step
Response あのとき、あＱはいった。

Input: きょうはいいてんきですね。
1/1 [==============================] - 0s 18ms/step
Response しょうせんのことばはしんだいのだから、このことばはしかしかれをみているのであった。

Input: すきなたべものは、なんですか。
1/1 [==============================] - 0s 16ms/step
Response ははおやはこういっていた。

Input: きょうは、さんぽにいきました。
1/1 [==============================] - 0s 19ms/step
Response やっぱりあるいていると、かれはこんどこそうのうえにまるところへいって、かのじょはかれにかえってきた。

Input: パソコンかスマホは、つかったことありますか？
1/1 [==============================] - 0s 16ms/step
Response それはそうとうのことばをもちゃげていった。



## ５モデル同士の会話

In [7]:
response_a = "こんにちは。"
response_b = ""
for i in range(100):
    print("魯迅A:", response_a)
    print()  
    vector_a = sentence_to_vector(response_a)
    
    response_b = respond(vector_a)
    print("魯迅B:", response_b)
    print()
    vector_b = sentence_to_vector(response_b)
    
    response_a = respond(vector_b)

魯迅A: こんにちは。

1/1 [==============================] - 0s 21ms/step


<ipython-input-3-891c0e32d576>:16: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  vector = np.zeros((1, max_length_x, n_char), dtype=np.bool)


ストリーミング出力は最後の 5000 行に切り捨てられました。
1/1 [==============================] - 0s 19ms/step
魯迅B: かれはいっぽんのうえにいっっていた。

1/1 [==============================] - 0s 20ms/step
魯迅A: かれはこういっていると、かれはこのときこんなにんをくっっていたが、それはいったいのかんじょうのなかにはなしをしていた。

1/1 [==============================] - 0s 19ms/step
魯迅B: そこでかれはこのときこういうにんをかんじた。

1/1 [==============================] - 0s 20ms/step
魯迅A: かれらはかれにかえってきた。

1/1 [==============================] - 0s 18ms/step
魯迅B: そうしてもういちどたっているので、それはいったいにんからあＱのまえにとびだしていた。

1/1 [==============================] - 0s 23ms/step
魯迅A: あＱはこのときこうふんをかんじた。

1/1 [==============================] - 0s 25ms/step
魯迅B: かれはいっしょにあいていると、かれはこんどこそろこうしょうをうけて、そのにんはかれにかわいそうにはかえってきた。

1/1 [==============================] - 0s 17ms/step
魯迅A: かれはひじょうにおぼえている。

1/1 [==============================] - 0s 22ms/step
魯迅B: そうしてもういちどゆるされた。

1/1 [==============================] - 0s 17ms/step
魯迅A: きっとあＱはいっしゅにあった。

1/1 [==============================] - 0s 19ms/step
魯迅B: かのじょは、それにもみないのかんがえていた。

1/1 [===================